In [213]:
%matplotlib inline

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error,r2_score
import fileinput
import operator
import random

#Fit the code to the screen
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

### Reading User Movie and Ratings

In [132]:
#Reading the movies data
my_path = !pwd
movies_filename = "/features/movie_features.pik"
user_filename = "/features/user_features_advanced.pik"
ratings_filename = "/ml-1m/ratings.dat"
movie_features = pickle.load(open(my_path[0]+movies_filename, 'rb'))
user_features = pickle.load(open(my_path[0]+user_filename, 'rb'))
user_features['gender'] = user_features['gender'].map({'F':0,'M':1})
dfratings = pd.read_csv(my_path[0]+ratings_filename, index_col=None, sep='::', header=None)
dfratings.columns = ["userId", "movieId", "rating", "timestamp"]
#dfratings.describe()

/tank/home/adif/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


In [211]:
#USER SIMILARITY_FUNCTIONS

N_MOST_SIMILAR=10
def cosine_similarity(user_features,userId1,userId2,similarity_features):
    user1 = user_features[user_features["userId"] == userId1][similarity_features]
    user2 = user_features[user_features["userId"] == userId2][similarity_features]
    sim = np.dot(user1,user2.T)/np.linalg.norm(user1)/np.linalg.norm(user2)
    return sim[0][0]
    

#returns the 'n' most similar users to 'userId' according to 'similarity_features'. 
#If 'predicted_movieId' is not 'None' we restrict the users to chose from only to users that rated the movie 
def n_most_similar(n,userId,user_features,similarity_features,predicted_movieId=None):
    #print user_features[user_features["userId"]==5530]
    #print user_features[user_features["userId"]==2106]
    users_to_chose_from = user_features
    if predicted_movieId!=None:
        users_to_chose_from = dfratings[dfratings["movieId"]==predicted_movieId]
    similarity_dict = dict()
    for simliarity_userId in users_to_chose_from["userId"].unique():
        if userId!=simliarity_userId:
            similarity_dict[simliarity_userId] = cosine_similarity(user_features,userId,simliarity_userId,similarity_features)
            #print simliarity_userId, similarity_dict[simliarity_userId]
    sorted_dict = sorted(similarity_dict.items(), key=operator.itemgetter(1)) 
    sorted_dict.reverse()
    ids =[]
    for items in sorted_dict[:min(n,len(sorted_dict)-1)]:
        ids.append(items[0])
    return ids


user_genre_pref_filename = '/explore/user_genre_pref.pik'
user_genre_pref = pickle.load(open(my_path[0]+user_genre_pref_filename, 'rb'))
similarity_features = list(user_genre_pref.columns)
similarity_feaures.append("rate_num")
similarity_feaures.append("rate_avg")
similarity_feaures.append("rate_std")  
similarity_feaures.append("avg_score_diff")
random_seed = 42
#print user_features["userId"][0]
MAX_SIMILAR_USERS=10
predictions = []

tested_userIds = [457,2768,1980,402,4126,3418,1112,3119,1737,1322]
#tested_userIds = [457,2768,1980]
ratings_pred = []
ratings_real = []
for n in range(1,MAX_SIMILAR_USERS):
    ratings_pred.append([])
    ratings_real.append([])    

for n in range(1,MAX_SIMILAR_USERS):
    for dummy_train_indices,dfratings_test_indices in dfratings_shuffle_indices:
        for tested_userId in tested_userIds:
            user_ratings = dfratings[dfratings["userId"]==tested_userId]
            user_ratings.reset_index(inplace=True)
            user_ratings_shuffle_indices = cross_validation.ShuffleSplit(len(user_ratings), n_iter=1, test_size=0.1, random_state=random_seed)
            for dummy_train_indices,user_ratings_test_indices in user_ratings_shuffle_indices:
                predicted_movieIds = (user_ratings["movieId"])[user_ratings_test_indices]
                for predicted_movieId in predicted_movieIds:
                    n_similar_userIds = n_most_similar(n,tested_userId,user_features,similarity_features,predicted_movieId)
                    similar_ratings = []
                    for similar_userId in n_similar_userIds:
                        similar_user_ratings = dfratings[dfratings["userId"]==similar_userId]
                        similar_movie_rating = similar_user_ratings[similar_user_ratings["movieId"] == predicted_movieId]["rating"]
                        similar_ratings.append(similar_movie_rating.item())
                    #user_rating = dfratings[dfratings["userId"]==tested_userId]
                    #user_movie_rating = user_rating[user_rating["movieId"] == predicted_movieId]["rating"]                    
                    predict = float(sum(similar_ratings))/float(len(similar_ratings))
                    ratings_pred[n-1].append(predict)
                    #ratings_real[n-1].append(user_movie_rating.item())
        with open(my_path[0]+'/ratings_pred{0}.pik'.format(n), 'wb') as handle:
            pickle.dump(ratings_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)                    
        #with open(my_path[0]+'/ratings_real{0}.pik'.format(n), 'wb') as handle:
            #pickle.dump(ratings_real, handle, protocol=pickle.HIGHEST_PROTOCOL)                                            
        print n,mean_squared_error(ratings_pred[n-1],ratings_real[0]),r2_score(ratings_pred[n-1],ratings_real[0])
                    
                    




1 1.48291571754 -0.25333959004
2 1.01025056948 -0.27116773905
3 0.902556314857 -0.33933298313
4 0.853929384966 -0.349503735692
5 0.799635535308 -0.378532237854
6 0.777145026576 -0.389450296171
7 0.770117614244 -0.436176654013
8 0.754093109339 -0.419425624428
9 0.753227031131 -0.481007291566


In [217]:
ratings_rand=[]
for i in range(len(ratings_real[0])):
    ratings_rand.append(random.randint(1,5))
print "random",mean_squared_error(ratings_rand,ratings_real[0]),r2_score(ratings_rand,ratings_real[0])    

random 3.69248291572 -0.779901852889


In [216]:
for n in range(10,15):
    ratings_pred.append([])
   

for n in range(10,15):
    for dummy_train_indices,dfratings_test_indices in dfratings_shuffle_indices:
        for tested_userId in tested_userIds:
            user_ratings = dfratings[dfratings["userId"]==tested_userId]
            user_ratings.reset_index(inplace=True)
            user_ratings_shuffle_indices = cross_validation.ShuffleSplit(len(user_ratings), n_iter=1, test_size=0.1, random_state=random_seed)
            for dummy_train_indices,user_ratings_test_indices in user_ratings_shuffle_indices:
                predicted_movieIds = (user_ratings["movieId"])[user_ratings_test_indices]
                for predicted_movieId in predicted_movieIds:
                    n_similar_userIds = n_most_similar(n,tested_userId,user_features,similarity_features,predicted_movieId)
                    similar_ratings = []
                    for similar_userId in n_similar_userIds:
                        similar_user_ratings = dfratings[dfratings["userId"]==similar_userId]
                        similar_movie_rating = similar_user_ratings[similar_user_ratings["movieId"] == predicted_movieId]["rating"]
                        similar_ratings.append(similar_movie_rating.item())
                    user_rating = dfratings[dfratings["userId"]==tested_userId]
                    user_movie_rating = user_rating[user_rating["movieId"] == predicted_movieId]["rating"]                    
                    predict = float(sum(similar_ratings))/float(len(similar_ratings))
                    ratings_pred[n-1].append(predict)
                    #ratings_real[n-1].append(user_movie_rating.item())
        with open(my_path[0]+'/ratings_pred{0}.pik'.format(n), 'wb') as handle:
            pickle.dump(ratings_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)                    
        #with open(my_path[0]+'/ratings_real{0}.pik'.format(n), 'wb') as handle:
            p#ickle.dump(ratings_real, handle, protocol=pickle.HIGHEST_PROTOCOL)                                            
        print n,mean_squared_error(ratings_pred[n-1],ratings_real[0]),r2_score(ratings_pred[n-1],ratings_real[0])
      

KeyboardInterrupt: 

In [131]:
ratings = dfratings["rating"][dfratings_test_indices]


1.83116883117


### Reading ratings data

In [4]:
#Reading the ratings data
ratings_filename = "/ml-1m/ratings.dat"
dfratings = pd.read_csv(my_path[0]+ratings_filename, index_col=None, sep='::', header=None)
dfratings.columns = ["userId", "movieId", "rating", "timestamp"]

/home/anat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


## Creating genres vector for each movie

In [5]:
#Disable false positive Pandas warning for 'SettingWithCopyWarning'
pd.options.mode.chained_assignment = None

#Adding genres vector to each movie
all_genres = ['Mystery', 'Sci-Fi', 'Crime', 'Drama', 'Animation', 'IMAX', 'Action', 'Comedy', 'Documentary', 'War', 'Romance', 'Horror', 
              'Film-Noir', 'Musical', 'Fantasy', 'Adventure', 'Children', 'Thriller', 'Western']
for genre in all_genres:
    dfmovies[genre] = dfmovies['genres'].apply(lambda x: x.find(genre)>=0).astype(int)

## Create users genre preference vectors

In [20]:
users = (dfratings["userId"]).unique()
genres_pref = []
for userId in users:
    user_ratings = dfratings[dfratings["userId"] == userId]
    movies_total = 0
    geners_rating_score = [0] * len(all_genres)
    for index, rating in user_ratings.iterrows():
        movieId = rating["movieId"]
        score = rating["rating"]
        movie_data = dfmovies.loc[dfmovies["movieId"] == movieId]
        for genre in all_genres:
            if (movie_data[genre].item() == 1):
                geners_rating_score[all_genres.index(genre)] += score
    genres_pref.append(geners_rating_score)
    if (userId % 1000 == 0):
        print userId

1000
2000
3000
4000
5000
6000


In [30]:
#Save the genres prefrence to dataframe and to pickle
user_pref_df = pd.DataFrame(genres_pref)
user_pref_df.columns = all_genres
with open(my_path[0]+'/explore/user_genre_pref.pik', 'wb') as handle:
    pickle.dump(user_pref_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open(my_path[0]+'/explore/user_genre_pref.pik', 'rb') as handle:
    user_pref_df = pickle.load(handle)

## Unify all the users features

### Reading the users data

In [32]:
users_filename = "/ml-1m/users.dat"
dfusers = pd.read_csv(my_path[0]+users_filename, sep='::', header=None)
dfusers.columns = ["userId","gender", "age", "occupation", "zip-code"]

/home/anat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [35]:
#Extracting gender, age and occupation features
del dfusers["zip-code"]
user_features = dfusers.join(user_pref_df)

In [21]:
#Calculating number of ratings and average rating per user
users = (dfratings["userId"]).unique()
user_rate_num = []
user_rate_avg = []
user_rate_std = []
for userId in users:
    user_ratings = dfratings[dfratings["userId"] == userId]
    user_rate_num.append(user_ratings.shape[0])
    user_rate_avg.append(np.average(user_ratings["rating"]))
    user_rate_std.append(np.std(user_ratings["rating"]))

In [25]:
user_features["rate_num"] = user_rate_num
user_features["rate_avg"] = user_rate_avg
user_features["rate_std"] = user_rate_std

In [28]:
with open(my_path[0]+'/features/user_features.pik', 'wb') as handle:
    pickle.dump(user_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
with open(my_path[0]+'/features/user_features.pik', 'rb') as handle:
    user_features = pickle.load(handle)

### Calculate movies features

In [6]:
#Calculate average rating and number of ratings
all_movies = dfmovies["movieId"]
avg_rate = []
num_rate = []
std_rate = []
for movie in all_movies:
    movie_data = dfratings[dfratings["movieId"] == movie]
    num_rate.append(movie_data.shape[0])
    avg_rate.append(np.average(movie_data["rating"]))
    std_rate.append(np.std(movie_data["rating"]))

/home/anat/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [7]:
#Removing movies with 0 reviews
movies_features = dfmovies.copy(deep=True)
exclude_movies = []
for i in sorted(all_movies.index, reverse=True):
    if (num_rate[i] == 0):
        exclude_movies.append(all_movies[i])
        del avg_rate[i]
        del num_rate[i]
        movies_features.drop(movies_features.index[i], inplace=True)

## Unify all movies features

In [15]:
del movies_features["genres"]
movies_features["avg_rate"] = avg_rate
movies_features["num_rate"] = num_rate
movies_features["std_rate"] = std_rate

In [19]:
with open(my_path[0]+'/features/movie_features.pik', 'wb') as handle:
    pickle.dump(movies_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [98]:
cross_validation.ShuffleSplit(len(user_features), n_iter=1, test_size=.01, random_state=random_seed)
user_features["userId"][userId_shuffle_index]

ShuffleSplit(6040, n_iter=1, test_size=0.01, random_state=42)